In [3]:
import numpy as np
from bayes_opt import BayesianOptimization

data = np.load('game_results/game_results_humans.npz')
xyz = data['xyz']
params = data['params']
meta = data['meta']

# Ground truth model in numpy
def gmm_model(params, x, y):
    # params: (N, 25), x/y: (N, 20)
    g = params[:24].reshape(6, 4)
    w, mx, my, s = g[:, 0], g[:,1], g[:, 2], g[:, 3]
    expo = -((x[...,None] - mx[None])**2 + (y[...,None] - my[None])**2) / s[None]
    res = np.sum(w[None] * np.exp(expo), axis=-1)
    
    return res * params[24]


# for i in range(len(params)):
#     assert np.allclose(gmm_model(params[i], xyz[i,:,0], xyz[i,:,1]), xyz[i,:,2])



In [ ]:
import numpy as np
from skopt.space import Integer
import skopt.space
from skopt import gp_minimize
from skopt.utils import use_named_args

# grid
x = np.linspace(0,1,100,endpoint=False)
grid = np.meshgrid(x,x)
search_space = [skopt.space.Integer(0, 99, name='x'), skopt.space.Integer(0, 99, name='y')]
#objective function
def f_objective(point):
    x,y=point
    return objective[x,y]

for person in xyz[:2]:
    for i,trial in enumerate(person):
        x0=[[int(x0[0]*100),int(x0[1]*100)] for x0 in xyz[i][:4]]

        for k,point in enumerate(trial):
            objective=-gmm_model(params[k], grid[0], grid[1])
            res=gp_minimize(f_objective, search_space, n_calls=20, 
                            x0=[[int(xy[0]*100),int(xy[1]*100)] for xy in trial[:len(x0)]],acq_func='EI')
            print(res['x_iters'])
            x0.append(res['x_iters'][-1])
            




IndexError: invalid index to scalar variable.

In [4]:
import numpy as np
x = np.linspace(0,1,100,endpoint=False)
grid = np.meshgrid(x,x)
def f_objective(point):
    x,y=point
    return objective[x,y]


from skopt.space import Integer
import skopt.space
from skopt import gp_minimize
from skopt.utils import use_named_args
search_space = [skopt.space.Integer(0, 99, name='x'), skopt.space.Integer(0, 99, name='y')]
for i in range(2,3):
    objective=-gmm_model(params[i], grid[0], grid[1])
    res=gp_minimize(f_objective, search_space, n_calls=20,n_initial_points=0, 
                    x0=[[int(x0[0]*100),int(x0[1]*100)] for x0 in xyz[i][:4]],acq_func='EI',
                    acq_optimizer='auto')


print(len(res['models']))
print(len(res['x_iters']))
print(len(res['func_vals']))
res

17
20
20


          fun: -0.7875563713633134
            x: [np.int64(76), np.int64(23)]
    func_vals: [-1.726e-08 -5.641e-03 ... -6.103e-01 -7.876e-01]
      x_iters: [[10, 10], [90, 10], [10, 90], [90, 90], [np.int64(90), np.int64(2)], [np.int64(89), np.int64(11)], [np.int64(92), np.int64(12)], [np.int64(85), np.int64(13)], [np.int64(82), np.int64(1)], [np.int64(86), np.int64(2)], [np.int64(83), np.int64(14)], [np.int64(98), np.int64(15)], [np.int64(80), np.int64(14)], [np.int64(78), np.int64(17)], [np.int64(77), np.int64(22)], [np.int64(68), np.int64(26)], [np.int64(79), np.int64(25)], [np.int64(72), np.int64(22)], [np.int64(80), np.int64(22)], [np.int64(76), np.int64(23)]]
       models: [GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                                        n_restarts_optimizer=2, noise='gaussian',
                                        normalize_y=True, random_state=944313539), GaussianProcessRegressor(kernel=1**2 

In [81]:
print(res['models'][0])

GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=2, noise='gaussian',
                         normalize_y=True, random_state=810545999)
